# Identifying Patient Cohorts in [MIMIC-II](http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3124312/)


In [ ]:
%matplotlib inline

In [ ]:
import pymysql
import pandas as pd
import getpass
import pandas as pd
import seaborn as sns
import datetime
import time
import matplotlib.pyplot as plt

In [ ]:
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

## Example Query: Heart Rate and Blood Pressure

#### Select a patient from the following ids
* 12613
* 11923
* 517
* 14898

In [ ]:
hr = pd.read_sql("""SELECT subject_id, 
                           icustay_id, 
                           charttime, 
                           realtime,
                           value1num,
                           value1uom
                    FROM chartevents
                    WHERE itemid in (211) AND
                          subject_id in (11923)"""
    ,conn)
hr.head()

In [ ]:
bp = pd.read_sql("""SELECT subject_id, 
                           icustay_id, 
                           charttime, 
                           realtime,
                           value1num,
                           value1uom,
                           value2num,
                           value2uom
                    FROM chartevents
                    WHERE itemid in (6, 51, 455, 6701) AND
                          subject_id in (11923)"""
    ,conn)
bp.head()

In [ ]:
print(bp["icustay_id"].value_counts())
print(bp["subject_id"].value_counts())

In [ ]:
hr["icustay_id"].value_counts()

### Plot Heart Rate as a Time Series

In [ ]:
hr.plot(x="realtime", y="value1num")

#### What went wrong?

As part of their de-identication process, the MIMIC2 creators have created large, random time shifts. We can look at the first data point to see what our calendar is

In [ ]:
print(hr["realtime"][0])


#### Our data is from the year 2654 which for whatever reason pandas can't plot.
### We can shift the data back to our own time
#### Compute the timedelta between the now and the first data point.

In [ ]:
first_time = hr["realtime"][0]
time_delta = first_time - datetime.datetime.now()
print(time_delta.days)

In [ ]:
first_time - time_delta

## Exercise: Use Pandas apply to create new columns with charttimes and real times shifted to starting at the current  date for the heart rate and blood pressure data

In [ ]:
hr[hr["icustay_id"]==14882].plot(x="new charttime", 
                                 y="value1num")
hr[hr["icustay_id"]!=14882].plot(x="new charttime", 
                                 y="value1num", color='red')

In [ ]:
bp[bp["icustay_id"]==14882].plot(x="new charttime", 
                                 y=["value1num", "value2num"])
bp[bp["icustay_id"]!=14882].plot(x="new charttime", 
                                 y=["value1num", "value2num"])

In [ ]:
hr["chart delta"] = \
hr.apply(lambda x: (x["charttime"] - 
                        x["realtime"]).total_seconds(), axis=1)
hr.head()

## Exercise: Timestamp from datetime
#### Using ``time.mktime`` and ``datetime.timetuple`` create timestamps for each measurement

In [ ]:
time.mktime(first_time.timetuple())

In [ ]:
hr[["new charttime","new realtime","chart delta"]][hr["icustay_id"]==14882].plot.line(x="new charttime", 
                               y="chart delta")